In [13]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

#  SparkSession 생성
spark = SparkSession \
    .builder \
    .appName("adam") \
    .getOrCreate()

# 외부 csv 데이터 불러온 후, 스키마 출력(inferSchema는 컬럼의 데이터 타입을 자동으로 추론)
data = spark.read.csv('./boston.csv', header=True, inferSchema=True)
data.printSchema()

# label인 medv를 제외한 컬럼을 가져오고 이를 하나의 피처 벡터 컬럼으로 변환 후 'features' 컬럼에 저장
feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)

# 학습, 테스트셋 분리
train_data, test_data = data.randomSplit([0.8, 0.2], seed=20230921)

root
 |-- _c0: integer (nullable = true)
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- black: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [45]:
# 모델 정의
rf = RandomForestRegressor(featuresCol="features", labelCol="medv")

# 파이프라인 정의
pipeline = Pipeline().setStages([rf])

# 하이퍼파라미터 후보군 정의
params = ParamGridBuilder().addGrid(rf.numTrees, [50, 100, 150]).build()


# evaluator 정의
evaluator = RegressionEvaluator() \
            .setMetricName("rmse") \
            .setPredictionCol("prediction") \
            .setLabelCol("medv")

# cross validator 정의
cv = CrossValidator()\
.setEstimator(pipeline)\
.setEvaluator(evaluator)\
.setEstimatorParamMaps(params)\
.setNumFolds(5)

model = cv.fit(train_data)

In [47]:
# 최적의 모델 가져오기
best_model = model.bestModel

# 최적의 모델의 하이퍼파라미터 값 확인
best_rf_model = best_model.stages[0]
best_rf_model

RandomForestRegressionModel: uid=RandomForestRegressor_0856a87da0d2, numTrees=100, numFeatures=14

In [48]:
# prediction 진행
predictions = model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="medv", predictionCol='prediction')
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print(f"rmse: {rmse:.4f}")
print(f"mae: {mae:.4f}")
print(f"r2: {r2:.4f}")

rmse: 3.7126
mae: 2.3035
r2: 0.8486
